# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 1:37PM,257169,10,0086370774,ISDIN Corporation,Released
1,Feb 10 2023 1:35PM,257168,19,TR-RG-02132023,"GCH Granules USA, Inc.",Released
2,Feb 10 2023 1:18PM,257167,15,30013851,"Alliance Pharma, Inc.",Executing
3,Feb 10 2023 1:18PM,257167,15,30013852,"Alliance Pharma, Inc.",Released
4,Feb 10 2023 1:17PM,257166,19,2022102868,"GUSA Granules USA, Inc.",Released
5,Feb 10 2023 1:12PM,257165,15,9141471,Lonza,Released
6,Feb 10 2023 12:57PM,257163,21,640949,"NBTY Global, Inc.",Released
7,Feb 10 2023 12:57PM,257162,21,640953,"NBTY Global, Inc.",Released
8,Feb 10 2023 12:46PM,257161,10,0086370703,ISDIN Corporation,Released
9,Feb 10 2023 12:44PM,257160,10,0086370699,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,257166,Released,1
34,257167,Executing,1
35,257167,Released,1
36,257168,Released,1
37,257169,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257165,NaN,NaN,1.0
257166,NaN,NaN,1.0
257167,NaN,1.0,1.0
257168,NaN,NaN,1.0
257169,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256949,40.0,1.0,6.0
257074,0.0,0.0,7.0
257080,0.0,0.0,5.0
257082,0.0,1.0,0.0
257089,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256949,40,1,6
257074,0,0,7
257080,0,0,5
257082,0,1,0
257089,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256949,40,1,6
1,257074,0,0,7
2,257080,0,0,5
3,257082,0,1,0
4,257089,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256949,40,1,6
1,257074,,,7
2,257080,,,5
3,257082,,1,
4,257089,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc."
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc."
4,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc."
5,Feb 10 2023 1:12PM,257165,15,Lonza
6,Feb 10 2023 12:57PM,257163,21,"NBTY Global, Inc."
7,Feb 10 2023 12:57PM,257162,21,"NBTY Global, Inc."
8,Feb 10 2023 12:46PM,257161,10,ISDIN Corporation
9,Feb 10 2023 12:44PM,257160,10,ISDIN Corporation
36,Feb 10 2023 12:44PM,257159,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation,,,1
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc.",,,1
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc.",,1,1
3,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc.",,,1
4,Feb 10 2023 1:12PM,257165,15,Lonza,,,1
5,Feb 10 2023 12:57PM,257163,21,"NBTY Global, Inc.",,,1
6,Feb 10 2023 12:57PM,257162,21,"NBTY Global, Inc.",,,1
7,Feb 10 2023 12:46PM,257161,10,ISDIN Corporation,,,1
8,Feb 10 2023 12:44PM,257160,10,ISDIN Corporation,,,27
9,Feb 10 2023 12:44PM,257159,10,ISDIN Corporation,,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation,1,,
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc.",1,,
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc.",1,1,
3,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc.",1,,
4,Feb 10 2023 1:12PM,257165,15,Lonza,1,,
5,Feb 10 2023 12:57PM,257163,21,"NBTY Global, Inc.",1,,
6,Feb 10 2023 12:57PM,257162,21,"NBTY Global, Inc.",1,,
7,Feb 10 2023 12:46PM,257161,10,ISDIN Corporation,1,,
8,Feb 10 2023 12:44PM,257160,10,ISDIN Corporation,27,,
9,Feb 10 2023 12:44PM,257159,10,ISDIN Corporation,21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation,1,,
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc.",1,,
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc.",1,1,
3,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc.",1,,
4,Feb 10 2023 1:12PM,257165,15,Lonza,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation,1.0,NaN,NaN
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc.",1.0,1.0,NaN
3,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Feb 10 2023 1:12PM,257165,15,Lonza,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 10 2023 1:37PM,257169,10,ISDIN Corporation,1.0,0.0,0.0
1,Feb 10 2023 1:35PM,257168,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
2,Feb 10 2023 1:18PM,257167,15,"Alliance Pharma, Inc.",1.0,1.0,0.0
3,Feb 10 2023 1:17PM,257166,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Feb 10 2023 1:12PM,257165,15,Lonza,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4371169,118.0,0.0,0.0
12,257082,0.0,1.0,0.0
15,1028424,9.0,2.0,40.0
16,2057060,8.0,0.0,0.0
19,771473,3.0,0.0,0.0
21,514325,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4371169,118.0,0.0,0.0
1,12,257082,0.0,1.0,0.0
2,15,1028424,9.0,2.0,40.0
3,16,2057060,8.0,0.0,0.0
4,19,771473,3.0,0.0,0.0
5,21,514325,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,118.0,0.0,0.0
1,12,0.0,1.0,0.0
2,15,9.0,2.0,40.0
3,16,8.0,0.0,0.0
4,19,3.0,0.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,118.0
1,12,Released,0.0
2,15,Released,9.0
3,16,Released,8.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,40.0,0.0,0.0,0.0
Executing,0.0,1.0,2.0,0.0,0.0,0.0
Released,118.0,0.0,9.0,8.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,40.0,0.0,0.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,0.0
2,Released,118.0,0.0,9.0,8.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,40.0,0.0,0.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,0.0
2,Released,118.0,0.0,9.0,8.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()